<a href="https://colab.research.google.com/github/mythrikulkarni/Aggie-Assist/blob/main/AggieAssist1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
with open(r"/content/train.json", "r") as read_file:
    train = json.load(read_file)

In [ ]:
train

[{'context': "There are many types of mental health professionals. The variety of providers and their services may be confusing. Each have various levels of education, training, and may have different areas of expertise. Finding the professional who best fits your needs may require some research. To find resources on campus for mental health, here's a helpful resource: https://shcs.ucdavis.edu/services/counseling-services",
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'What is the difference between mental health professionals?',
    'answers': [{'text': "There are many types of mental health professionals. The variety of providers and their services may be confusing. Each have various levels of education, training, and may have different areas of expertise. Finding the professional who best fits your needs may require some research. To find resources on campus for mental health, here's a helpful resource: https://shcs.ucdavis.edu/services/counseling-services",

In [ ]:
with open(r"/content/test.json", "r") as read_file:
    test = json.load(read_file)

In [ ]:
test

[{'context': 'The Student Health and Counseling Services (SHCS) provides a range of mental health services, including individual counseling, group therapy, psychiatric medication management, and crisis support. These services are available to all registered students, and are covered by the student health insurance plan (UC SHIP) and have prices listed on their website for students under another health insurance plan: https://shcs.ucdavis.edu/services/primary-care',
  'qas': [{'id': '00041',
    'is_impossible': False,
    'question': 'What mental health services are provided by the SHCS?',
    'answers': [{'text': 'The Student Health and Counseling Services (SHCS) provides a range of mental health services, including individual counseling, group therapy, psychiatric medication management, and crisis support.',
      'answer_start': 0}]}]},
 {'context': 'Individual Counseling is confidential short-term therapy available to all registered UC Davis students for free at North Hall. UC Davi

In [ ]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 40
model_args.evaluate_during_training = True
model_args.n_best_size=1
model_args.num_train_epochs=1

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 1,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3, 
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [ ]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args, use_cuda= False
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [ ]:
!rm -rf outputs


In [ ]:
model.train_model(train, eval_data=test)

add example index and unique id: 100%|██████████| 39/39 [00:00<00:00, 232354.91it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

wandb: Currently logged in as: rdas. Use `wandb login --relogin` to force relogin


Running Epoch 0 of 1:   0%|          | 0/1 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 41/41 [00:00<00:00, 97.36it/s]

add example index and unique id: 100%|██████████| 41/41 [00:00<00:00, 97708.22it/s]


Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(1,
 {'global_step': [1],
  'correct': [0],
  'similar': [28],
  'incorrect': [12],
  'train_loss': [4.871259689331055],
  'eval_loss': [-0.22613976895809174]})

In [ ]:
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
with open(r"/content/predictions.json", "r") as read_file:
    to_predict = json.load(read_file)

In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 18/18 [00:00<00:00, 34426.57it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '00016', 'answer': ['it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which', 'services are being', 'it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which information is worth considering and which']}, {'id': '00017', 'answer': ['map to follow towards']}, {'id': '00018', 'answer': ['empty']}, {'id': '00019', 'answer': ['empty']}, {'id': '00020', 'answer': ['ercising,', 'ercising, spending time with friends or family, practicing mindfulness or meditation, and listening to music or podcasts. The goal is to shift your focus away from negative thoughts and feelings']}, {'id': '00021', 'answer': ['for consumers, their family members and friends. Some support groups are peer-led, while others may be led by a mental health professional. Information about UC Davis Group Services can be found here', 'for consumers, their family members and friends. Some support groups are peer-led, while others may be led

In [ ]:
to_predict = [
    {
        "context": "Having little to no energy and Feeling hopeless and helpless, Overusing substances like alcohol or cigarettes, Experiencing hallucinations or uncharacteristic thoughts/behaviors, Thinking about harming or killing yourself or others, Inability to perform daily tasks like studying or biking to school, Experiencing severe mood swings, Having unexplained aches or pains Ares several symptoms that you might be struggling with you mental health. If you would like to recieve help you can call 911 or go to the nearest emergency room. UC Davis SHCS offers crisis counseling services and has an after-hours crisis line at 530-752-2349. You can also contact the National Suicide Prevention Lifeline at 1-800-273-TALK (8255) or use their online chat feature. https://mentalhealth.ucdavis.edu/resources/signs-symptoms-statistics",
        "qas": [
            {
                "question": "What are some indicators that I am struggling with my mental health?",
                "id": "0",
            }
        ],
    }
]

In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)
     

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2577.94it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['no energy and Feeling hopeless and helpless, Overusing substances like alcohol or cigarettes, Experiencing hallucinations or uncharacteristic thoughts/behaviors, Thinking about harming or killing yourself or others, Inability to', 'no energy and Feeling hopeless and helpless, Overusing substances like alcohol or cigarettes, Experiencing hallucinations or uncharacteristic thoughts/behaviors, Thinking about harming or killing yourself or others, Inability to perform daily tasks like studying or biking to school, Exper']}]
